In [1]:
import numpy as np 
import torch
import torch.nn as nn
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
ds = pd.read_csv("diabetes.csv") #loading the dataset using pandas , so ds is a pandas dataframe

In [3]:
X = ds.iloc[:,0:-1].values
y = ds.iloc[: , -1].values  

In [4]:
print(type(X))
print(type(y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
print(y)
y = le.fit_transform(y)
print(y)

['positive' 'negative' 'positive' 'negative' 'positive' 'negative'
 'positive' 'negative' 'positive' 'positive' 'negative' 'positive'
 'negative' 'positive' 'positive' 'positive' 'positive' 'positive'
 'negative' 'positive' 'negative' 'negative' 'positive' 'positive'
 'positive' 'positive' 'positive' 'negative' 'negative' 'negative'
 'negative' 'positive' 'negative' 'negative' 'negative' 'negative'
 'negative' 'positive' 'positive' 'positive' 'negative' 'negative'
 'negative' 'positive' 'negative' 'positive' 'negative' 'negative'
 'positive' 'negative' 'negative' 'negative' 'negative' 'positive'
 'negative' 'negative' 'positive' 'negative' 'negative' 'negative'
 'negative' 'positive' 'negative' 'negative' 'positive' 'negative'
 'positive' 'negative' 'negative' 'negative' 'positive' 'negative'
 'positive' 'negative' 'negative' 'negative' 'negative' 'negative'
 'positive' 'negative' 'negative' 'negative' 'negative' 'negative'
 'positive' 'negative' 'negative' 'negative' 'positive' 'negat

In [6]:
y = np.array(y, dtype = 'float64')

In [7]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [8]:
X = torch.tensor(X)
y = torch.tensor(y)

In [9]:
y = y.unsqueeze(1)

In [10]:
class Dataset(Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def __getitem__(self,index):
        return self.X[index] , self.y[index]
    def __len__(self):
        return len(self.X)

In [11]:
dataset = Dataset(X,y)
len(dataset)
print(dataset[0]) 

(tensor([ 0.6399,  0.8483,  0.1496,  0.9073, -0.6929,  0.2040,  1.4260],
       dtype=torch.float64), tensor([1.], dtype=torch.float64))


In [12]:
trainLoader = torch.utils.data.DataLoader(dataset = dataset , batch_size = 32 , shuffle = True)

In [13]:
print("There is {} batches in the dataset".format(len(trainLoader)))
for (x,y) in trainLoader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(x.shape))
    print("Labels:  {}".format(y.shape))
    break


There is 24 batches in the dataset
For one iteration (batch), there is:
Data:    torch.Size([32, 7])
Labels:  torch.Size([32, 1])


In [14]:
class Model(nn.Module):
    def __init__(self, inputFeatures , outputFeatures):
        super(Model , self).__init__()
        self.fc1 = nn.Linear(inputFeatures , 5)
        self.fc2 = nn.Linear(5 , 4)
        self.fc3 = nn.Linear(4 , 3)
        self.fc4 = nn.Linear(3 , outputFeatures)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
    def forward(self,X):
        out = self.fc1(X)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out 

In [15]:
net = Model(7,1)
criterion = torch.nn.BCELoss(size_average  = True)
optimizer = torch.optim.SGD(net.parameters() , lr = 0.1 , momentum = 0.9 )

C:\Users\harsh\anaconda3\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [16]:
epochs = 200
for epoch in range(200):
    for inputs,labels in trainLoader:
        inputs = inputs.float()
        labels = labels.float()
        outputs = net(inputs) #this is forward prop
        loss = criterion(outputs , labels)
        optimizer.zero_grad() # without causing accumulation it sets to zero
        loss.backward() #computes back prop gradients 
        optimizer.step() #updates grapdients here 
    outputs = (outputs > 0.5).float()
    accuracy = (outputs == labels).float().mean()
    print("Epoch {}/{}, Loss : {:.3f} , Accuracy : {:.3f}".format(epoch +1 , epochs , loss , accuracy))

Epoch 1/200, Loss : 0.593 , Accuracy : 0.688
Epoch 2/200, Loss : 0.333 , Accuracy : 0.844
Epoch 3/200, Loss : 0.555 , Accuracy : 0.688
Epoch 4/200, Loss : 0.383 , Accuracy : 0.812
Epoch 5/200, Loss : 0.449 , Accuracy : 0.781
Epoch 6/200, Loss : 0.868 , Accuracy : 0.531
Epoch 7/200, Loss : 0.553 , Accuracy : 0.656
Epoch 8/200, Loss : 0.393 , Accuracy : 0.812
Epoch 9/200, Loss : 0.496 , Accuracy : 0.656
Epoch 10/200, Loss : 0.400 , Accuracy : 0.781
Epoch 11/200, Loss : 0.500 , Accuracy : 0.719
Epoch 12/200, Loss : 0.423 , Accuracy : 0.781
Epoch 13/200, Loss : 0.587 , Accuracy : 0.719
Epoch 14/200, Loss : 0.684 , Accuracy : 0.688
Epoch 15/200, Loss : 0.356 , Accuracy : 0.781
Epoch 16/200, Loss : 0.554 , Accuracy : 0.750
Epoch 17/200, Loss : 0.470 , Accuracy : 0.844
Epoch 18/200, Loss : 0.498 , Accuracy : 0.750
Epoch 19/200, Loss : 0.389 , Accuracy : 0.781
Epoch 20/200, Loss : 0.476 , Accuracy : 0.812
Epoch 21/200, Loss : 0.503 , Accuracy : 0.688
Epoch 22/200, Loss : 0.509 , Accuracy : 0.7